In [1]:
import pandas as pd
import json
import os # Operaciones relacionadas con el sistema de archivos
import datetime

### Metadatos

In [2]:
# Función para leer y filtrar los archivos JSON
def filtrar_archivos_json(directorio, categoria_busqueda):
    ''' Esta función, llamada filtrar_archivos_json, se encarga de buscar y filtrar 
        archivos JSON en un directorio específico en función de una categoría de búsqueda 
        proporcionada.   '''
    resultados = []
    for archivo in os.listdir(directorio):
            ruta_archivo = os.path.join(directorio, archivo)

            # Leer el contenido del archivo JSON
            with open(ruta_archivo) as f:
                archivo_json = f.readlines()

            # Cargar archivo JSON linea por linea
            for linea in archivo_json:
                data = json.loads(linea)
                # Verificar si la categoría buscada está presente en el archivo JSON
                categorias = data.get("category", [])
                if isinstance(categorias, list) and any(categoria_busqueda.lower() in categoria.lower() for categoria in categorias):
                    resultados.append(data)
    return resultados


In [3]:
# Filtrar los archivos JSON para la categoría "restaurant"
restaurant = filtrar_archivos_json('../PROYECTO_FINAL_EQUIPO/Google/metadata-sitios', "restaurant")
# Convertir los resultados a un pandas DataFrame
df_restaurant = pd.DataFrame(restaurant)

In [4]:
df_restaurant.shape

(121637, 15)

In [5]:
df_restaurant.sample(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
54705,Smokey Joes Kosher BBQ,"Smokey Joes Kosher BBQ, 494 Cedar Ln, Teaneck,...",0x89c2f0a5f0e15573:0xea2514f01b581e68,"Airy eatery for wood-fired, pit-smoked, glatt ...",40.889484,-74.023518,"[Barbecue restaurant, Kosher restaurant, Mexic...",3.3,38,$$,"[[Saturday, Closed], [Sunday, 11:30AM–9:30PM],...","{'Service options': ['Delivery'], 'Planning': ...",Permanently closed,"[0x89c2f1a70f55a4bd:0xae64b1ca30df3797, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...
53800,El Molina Rojo,"El Molina Rojo, 1108 Caldwell Blvd, Nampa, ID ...",0x54ae4dbdb45af14f:0x65714064c2426892,None,43.600058,-116.594451,[Mexican restaurant],5.0,2,None,None,"{'Service options': ['Takeout', 'Dine-in', 'De...",None,None,https://www.google.com/maps/place//data=!4m2!3...


In [19]:
#1061 restaurantes duplicados
df_restaurant['gmap_id'].duplicated().sum()

1061

In [20]:
#Revisión de restaurantes duplicados
restaurantes_duplicados = df_restaurant[df_restaurant['gmap_id'].duplicated(keep=False)]
# Ordena los restaurantes duplicados por gmap_id
restaurantes_duplicados_ordenados = restaurantes_duplicados.sort_values(by='gmap_id')
restaurantes_duplicados_ordenados.head(2)


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
36445,Bar 91,"Bar 91, 400 Ave of the Champions, Palm Beach G...",0x146c6711a70d7991:0xe478335f760c8ccc,None,26.828991,-80.141201,"[Bar & grill, Bar, Restaurant, Sports bar]",3.9,8,None,"[[Wednesday, 6:30AM–9PM], [Thursday, 6:30AM–9P...","{'Service options': ['Outdoor seating', 'Takeo...",Closed ⋅ Opens 6:30AM Thu,"[0x88ded52e4b87e21d:0x773ff6613ec82a36, 0x88de...",https://www.google.com/maps/place//data=!4m2!3...
37506,Bar 91,"Bar 91, 400 Ave of the Champions, Palm Beach G...",0x146c6711a70d7991:0xe478335f760c8ccc,None,26.828991,-80.141201,"[Bar & grill, Bar, Restaurant, Sports bar]",3.9,8,None,"[[Wednesday, 6:30AM–9PM], [Thursday, 6:30AM–9P...","{'Service options': ['Outdoor seating', 'Takeo...",Closed ⋅ Opens 6:30AM Thu,"[0x88ded52e4b87e21d:0x773ff6613ec82a36, 0x88de...",https://www.google.com/maps/place//data=!4m2!3...


In [21]:
#Eliminar duplicados
df_restaurant = df_restaurant.drop_duplicates(subset='gmap_id') 

In [22]:
#Finalmente tenemos 120,575 restaurantes con 15 atributos.
df_restaurant.shape

(120576, 15)

#### Reviews-estados

In [87]:
ruta_reviews_estados = "../PROYECTO_FINAL_EQUIPO/Google/reviews-estados/"
rutas_subfolders = [f.path for f in os.scandir(ruta_reviews_estados) if f.is_dir()]

In [88]:
def df_json(ruta_folder):
    '''Unión de archivos en un DataFrame '''
# Inicializa una lista para almacenar los objetos JSON de los archivos
    data = []
    
    # Itera a través de los archivos en la carpeta especificada
    for file_name in os.listdir(ruta_folder):
        # Construye la ruta completa de cada archivo
        file_path = os.path.join(ruta_folder, file_name)
        # Abre el archivo en modo lectura ('r')
        with open(file_path, 'r') as file:
            # Itera a través de las líneas del archivo
            for line in file:
                # Elimina espacios en blanco al principio y al final de la línea
                line = line.strip()
                # Intenta cargar la línea como un objeto JSON
                json_obj = json.loads(line)
                    # Agrega el objeto JSON a la lista de datos
                data.append(json_obj)

    # Crea un DataFrame de pandas a partir de la lista de datos
    df = pd.DataFrame(data)
    
    # Devuelve el DataFrame resultante
    return df

In [89]:
def df_json_general(subfolders):
    # Inicializa una lista para almacenar DataFrames individuales
    df_list = []
    
    # Itera a través de las subcarpetas especificadas
    for subfolder in subfolders:
        # Llama a la función read_json_files para procesar los archivos JSON en la subcarpeta
        df = df_json(subfolder)
        
        # Agrega el DataFrame resultante a la lista
        df_list.append(df)
    
    # Devuelve una lista de DataFrames, uno por cada subcarpeta
    return df_list

In [90]:

lista_df = df_json_general(rutas_subfolders)

In [91]:
df_reviews_res = pd.concat(lista_df, ignore_index=True)

In [92]:
df_reviews_res.shape

(8879925, 8)

Nos quedamos únicamente con las Reviews de Restaurantes. 

In [93]:
df_Restaurantes_Review = pd.merge(df_reviews_res, df_restaurant, on=["gmap_id"])

In [94]:
#Finalmente tenemos 152,797 Reviews entre nuestros restaurantes para Google.

df_Restaurantes_Review.shape

(152797, 22)

In [95]:
df_Restaurantes_Review.head(2)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,name_y,address,...,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,104959831058524862747,Samuel Benson,1630018420109,4,A pretty seamless experience from top to botto...,None,None,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,Wingstop,"Wingstop, 2275 N University Pkwy, Provo, UT 84604",...,-111.669659,[Chicken wings restaurant],4.0,17,None,"[[Wednesday, 11AM–12AM], [Thursday, 11AM–12AM]...","{'Service options': ['Delivery', 'Takeout', 'D...",Open ⋅ Closes 12AM,None,https://www.google.com/maps/place//data=!4m2!3...
1,100925653611153628883,Emily Probst,1629223016683,5,Great food and great service! Being so close t...,None,None,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,Wingstop,"Wingstop, 2275 N University Pkwy, Provo, UT 84604",...,-111.669659,[Chicken wings restaurant],4.0,17,None,"[[Wednesday, 11AM–12AM], [Thursday, 11AM–12AM]...","{'Service options': ['Delivery', 'Takeout', 'D...",Open ⋅ Closes 12AM,None,https://www.google.com/maps/place//data=!4m2!3...


In [96]:
df_Restaurantes_Review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152797 entries, 0 to 152796
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           152797 non-null  object 
 1   name_x            152797 non-null  object 
 2   time              152797 non-null  int64  
 3   rating            152797 non-null  int64  
 4   text              95476 non-null   object 
 5   pics              5970 non-null    object 
 6   resp              18174 non-null   object 
 7   gmap_id           152797 non-null  object 
 8   name_y            152797 non-null  object 
 9   address           152726 non-null  object 
 10  description       81106 non-null   object 
 11  latitude          152797 non-null  float64
 12  longitude         152797 non-null  float64
 13  category          152797 non-null  object 
 14  avg_rating        152797 non-null  float64
 15  num_of_reviews    152797 non-null  int64  
 16  price             10

Normalización de columna "Time"

In [102]:
def convertir_a_datetime(timestamp_ms):
    fecha_hora = datetime.datetime.fromtimestamp(timestamp_ms / 1000.0)  # Dividir por 1000 para convertir a segundos
    return fecha_hora

# Aplicar la función a la columna 'time' y guardar el resultado en la misma columna
df_Restaurantes_Review['time'] = df_Restaurantes_Review['time'].apply(convertir_a_datetime)

# Ordenar el DataFrame por la columna 'time' de menor a mayor
inner_merged = df_Restaurantes_Review.sort_values(by='time')

In [103]:
inner_merged['time']

127049   2005-06-09 19:00:00.000
97210    2007-08-27 17:59:02.178
93689    2007-10-27 21:16:16.877
58463    2007-11-24 09:54:21.134
32200    2007-11-24 23:31:04.091
                   ...          
140      2021-09-05 23:25:25.493
50901    2021-09-06 12:26:28.948
50891    2021-09-06 12:26:28.948
137923   2021-09-06 21:18:30.702
137903   2021-09-06 21:18:30.702
Name: time, Length: 152797, dtype: datetime64[ns]

In [104]:
inner_merged[ inner_merged['time']>'2015-11-24'].shape

(150335, 22)